In [1]:
from phySyncFirmata import ArduinoNano, util
import time

# declare board

board = ArduinoNano('/dev/ttyUSB0')

In [2]:
# reset all outputs

for i in set(range(2, 13)) - {10, 11}:
    board.digital[i].write(1)

In [3]:
# define led pins

leds = [board.get_pin('d:' + str(i) + ':o') for i in range(2, 10)]

# define switch pins

switches = [board.get_pin('d:' + str(j) + ':i') for j in range(10, 12)]

switches.extend([board.get_pin('a:' + str(k) + ':i') for k in range(7, 1, -1)])

In [4]:
# start iterators

it = util.Iterator(board)
it.start()

In [5]:
# pong helper function - period: time between successive LED activations, direction: 0 if forward, 1 if backward
# threshold: waiting time for user to press button as soon as LED wave reaches extreme LED

def pong_helper(period, direction, threshold):
    if direction == 0:
        r = list(range(len(leds)))
    else:
        r = list(range(len(leds) - 1, -1, -1))
    for i in r[:-1]:
        leds[i].write(0)
        board.pass_time(period)
        leds[i].write(1)
    leds[r[-1]].write(0)
    success = False
    start = time.time()
    while time.time() - start < threshold:
        if bool(round(float(switches[r[-1]].read()))) == False:
            success = True
            break
        board.pass_time(0.01)
    leds[r[-1]].write(1)
    return success

In [12]:
# play pong game - a 'wave' propogates through LEDS and the user(s) is/are expected to press the switch within 
# 'threshold' seconds when the wave reaches one of the extreme LEDs to reflect it. Waves progressively become
# faster, and threshold keeps on decreasing.

threshold = 1

game_ended = False

period = 0.5

cycles_to_increase_wave_speed = 3

cycles_to_decrease_reaction_time = 10

count = 0

for i in leds:
    i.write(1)

print("Game starting in 2 seconds. Wave will propogate from player 1's end", end = "\n\n")
    
time.sleep(2)
    
while not game_ended:
    game_ended = not pong_helper(period, count % 2, threshold)
    if count > 1 and count % cycles_to_increase_wave_speed == 0:
        period = period / 1.5
    if count > 1 and count % cycles_to_decrease_reaction_time == 0:
        threshold = threshold / 1.1
    count += 1

count -= 1

p1_score = count // 2 if count % 2 == 1 else count - count // 2
p2_score = count - p1_score

print("Player 1:", p1_score, "Player 2:", p2_score, end = "\n\n")
print(["Player 1", "Player 2"][p1_score < p2_score], "wins! Your LED is on now!")

# when wave ends, scores of both players are printed. The extreme LED on the side of the player who wins
# is turned on

if count % 2 == 0:
    leds[0].write(0)
else:
    leds[-1].write(0)

Game starting in 2 seconds. Wave will propogate from player 1's end

Player 1: 1 Player 2: 1

Player 1 wins! Your LED is on now!


In [68]:
# exit

board.exit()